<a href="https://colab.research.google.com/github/returaj/cs6910/blob/assginment1_akash/pred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/drive')
!cp drive/MyDrive/DLASSIGNMENT/optimizer.py .
import optimizer
!cp drive/MyDrive/DLASSIGNMENT/activation_func.py .
import activation_func
!cp drive/MyDrive/DLASSIGNMENT/PREDICTION.py .
import PREDICTION
!cp drive/MyDrive/DLASSIGNMENT/loss_func.py .
import loss_func
##!cp drive/MyDrive/DLASSIGNMENT/feed_forward_nn.py .
##import feed_forward_nn

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
##!ls "/content/drive/My Drive/DLASSIGNMENT"
##!python3 "/content/drive/My Drive/DLASSIGNMENT/feed_forward_nn.py"

activation_func.py  loss_func.py  PREDICTION.py
feed_forward_nn.py  optimizer.py  __pycache__
4431872/4422102 [==============================] - 0s 0us/step
epoch: 1, error: 2.6596374901436577
epoch: 2, error: 2.599343850809251
epoch: 3, error: 2.550354399331724
epoch: 4, error: 2.5099563139212417
epoch: 5, error: 2.47659916134711
epoch: 6, error: 2.44868875121824
epoch: 7, error: 2.4252983166873476
epoch: 8, error: 2.4053218345128062
epoch: 9, error: 2.387860338813435
epoch: 10, error: 2.3732790666497015


In [6]:
%%capture
!pip install wandb
import wandb

In [8]:
from wandb.keras import WandbCallback
wandb.login()

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [67]:
import numpy as np
from keras.datasets import fashion_mnist
from optimizer import SGD,MGD,NESTGD,ADAM,NADAM,ADAGRAD,RMSPROP
from activation_func import Sigmoid, Relu, Tanh
from loss_func import CrossEntropy, MinSquaredError
from PREDICTION import ACCURACY
from sklearn.model_selection import train_test_split
np.random.seed(2)
activation_functions ={"sigmoid":Sigmoid(),"tanh":Tanh(),"Rectified_unit":Relu}
loss= {"SquaredError":MinSquaredError(), "CrossEntropy":CrossEntropy()}
##optimizers={"stochastic_gd":SGD(),"momentum_gd":MGD(),"nest_gd":NESTGD(),"rmsprop_gd":RMSPROP()}
class FNN(object):
  def __init__(self,input_size,output_size,hidden_layers_size,optimizer,batch_size,activation_func,loss_function,rate,wand_b=True,init='random',reg=0):
    self.input_size = input_size
    self.output_size = output_size
    self.weight, self.bias = None, None
    self.act_func = activation_functions[activation_func]
    self.loss_func = loss[loss_function]
    self.reg = reg
    self.wand_b=wand_b
    self.optimizer=optimizer
    self.hidden_layers_size=hidden_layers_size
    self.init=init
    self.batch_size=batch_size
    self.rate=rate
    self.initialize(self.input_size,self.hidden_layers_size,self.output_size,self.init.lower())

  def initialize(self, input_size, hidden_layers_size, output_size, type):
    self.weight, self.bias = [], []
    prev_layer_size = input_size
    hidden_layers_size.append(output_size)
    for curr_layer_size in hidden_layers_size:
      std = np.sqrt(prev_layer_size * curr_layer_size) if type == 'xavier' else 1
      self.weight.append(np.random.rand(prev_layer_size, curr_layer_size)/std)
      self.bias.append(np.zeros(curr_layer_size))
      prev_layer_size = curr_layer_size

  @staticmethod
  def softmax(x):
    max_x = np.max(x, axis=1, keepdims=True)
    exp_prob = np.exp(x - max_x)
    prob = exp_prob / np.sum(exp_prob, axis=1, keepdims=True)
    return prob

  def forward(self, X):
    layer_output = []
    prev_layer = X
    num_hidden_layers = last_layer = len(self.weight) - 1
    for t in range(num_hidden_layers):
      w, b = self.weight[t], self.bias[t]
      next_layer = self.act_func.apply(np.dot(prev_layer, w) + b)
      layer_output.append(next_layer)
      prev_layer = next_layer
    w, b = self.weight[last_layer], self.bias[last_layer]
    prob = self.softmax(np.dot(prev_layer, w) + b)
    layer_output.append(prob)
    return layer_output

  def backward(self, X, y, layer_output):
    batch_size, _ = X.shape
    num_hidden_layers = last_layer = len(layer_output)-1
    dw, db = [None]*(num_hidden_layers+1), [None]*(num_hidden_layers+1)
    for t in range(num_hidden_layers, -1, -1):
      if t == last_layer:
        dh = self.loss_func.grad(layer_output[t], y)
      else:
        dh = np.dot(dh_fwd, self.weight[t+1].T) * self.act_func.grad(layer_output[t])
      prev_layer_output = X if t==0 else layer_output[t-1]
      dw[t] = np.dot(prev_layer_output.T, dh) + self.reg * self.weight[t]
      db[t] = np.sum(dh, axis=0)
      dh_fwd = dh
    return dw, db

  def optimization(self,epochs,model,optimizer):
    (x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
    x_train,x_val,y_train,y_val=train_test_split(x_train, y_train, test_size=0.1, random_state=42)
    print(y_train.shape)
    consider = 1000
    model=model
    X = np.array([(x_train[i].flatten())/255 for i in range(consider)]) 
    #print(X.shape)
    X_val= np.array([x_val[i].flatten() for i in range(consider)]) / 255
    Y = y_train[:consider]
    #print(Y.shape)
    Y_val = y_val[:consider]
    accuracy1=[]
    accuracy2=[]
    err=[]
    optimizers={"stochastic_gd":SGD,"momentum_gd":MGD,"nest_gd":NESTGD,"rmsprop_gd":RMSPROP,
                "adam_gd":ADAM,"nadam_gd":NADAM,"adagrad_gd":ADAGRAD}
  ##model = FNN(784,10,hidden_layers, act_func=act_func, loss_func=loss_func, init='random')
    optimize=optimizers[optimizer](model,self.rate)
    acc=ACCURACY()
    for ep in range(1, epochs+1):
      ids = np.arange(consider)
      np.random.shuffle(ids)
      start, end = 0,self.batch_size
      while end > start:
        x, y = X[ids[start:end]], Y[ids[start:end]]
        optimize.optimize(x, y)
        start, end = end, min(consider, end+self.batch_size)
      err = self.loss_func.error(X,Y,model)
      accuracy1.append(acc.predict(X,Y,model))
      accuracy2.append(acc.predict(X_val,Y_val,model))
      wandb.log({ "validationaccuracy":np.mean(accuracy2),"trainingaccuracy":np.mean(accuracy1),"epochs":ep})
      print(f'epoch: {ep} error: {err}')
    return accuracy2

In [68]:
def layers(hidden_layers,hidden_per_node):
  hidden=[]
  for i in range(hidden_layers):
    hidden.append(hidden_per_node)
  return hidden

In [69]:
##(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
##(x_train,y_train),(x_val,y_val)=train_test_split(x_train, y_train, test_size=0.1, random_state=42)
##consider=100
##X = np.array([(x_train[i].flatten())/255 for i in range(consider)])
##Y = y_train[:consider]
sweep_config = {
  "name": "Bayesian Sweep",
  "method": "bayes",
  "metric":{
  "name": "validationaccuracy",
  "goal": "maximize"
  },
  "parameters":{"max_epochs": {"values": [10, 50]}, 
                "hidden_layers": {"values": [3, 4, 5]}, 
                 "hidden_node_layer": {"values": [32, 64, 128]}, 
                "rate": {"values": [1e-3, 1e-4]}, 
                "optimizer": {"values": ["stochastic_gd","momentum_gd","adagrad_gd","rmsprop_gd","adam_gd","nadam_gd"]}, 
                "batch_size": {"values": [16,128, 1024]}, 
                "weight_initialization": {"values": ['random', 'xavier']} , 
                "activation_function": {"values": ["sigmoid", "tanh", "Rectified_unit"]}, 
                "loss_function": {"values": ["CrossEntropy", "SquaredError"]}, 
                  }}
#sweep_id=wandb.sweep(sweep_config, project = "1")
def train(config=sweep_config):
  
  with wandb.init(config = config):
    config = wandb.init().config
    wandb.run.name="e_{}_hl_{}_opt_{}_bs_{}_init_{}_ac_{}_loss_{}".format(config.max_epochs,
                                                                      config.hidden_node_layer,
                                                                      config.optimizer,
                                                                      config.batch_size,
                                                                      config.weight_initialization,
                                                                      config.activation_function,
                                                                      config.loss_function)
    no_hidden_layers=layers(config.hidden_layers,config.hidden_node_layer)
    nn_model = FNN(784,10,no_hidden_layers,config.optimizer,config.batch_size,config.activation_function,
                   config.loss_function,config.rate
                                 ,True,config.weight_initialization)
    accuracy2=nn_model.optimization(config.max_epochs,nn_model,config.optimizer)
sweep_id=wandb.sweep(sweep_config, project = "1")
wandb.agent(sweep_id,train,count = 5)
        
       
        

Create sweep with ID: 0owc7afy
Sweep URL: https://wandb.ai/akashsainics21m003/1/sweeps/0owc7afy


wandb: Agent Starting Run: ija9hgj9 with config:
wandb: 	activation_function: Rectified_unit
wandb: 	batch_size: 16
wandb: 	hidden_layers: 3
wandb: 	hidden_node_layer: 128
wandb: 	loss_function: CrossEntropy
wandb: 	max_epochs: 10
wandb: 	optimizer: nadam_gd
wandb: 	rate: 0.0001
wandb: 	weight_initialization: xavier


wandb: ERROR Run ija9hgj9 errored: NameError("name 'model' is not defined")


(54000,)


wandb: Agent Starting Run: u9296m2v with config:
wandb: 	activation_function: tanh
wandb: 	batch_size: 128
wandb: 	hidden_layers: 5
wandb: 	hidden_node_layer: 128
wandb: 	loss_function: CrossEntropy
wandb: 	max_epochs: 10
wandb: 	optimizer: adam_gd
wandb: 	rate: 0.0001
wandb: 	weight_initialization: random


wandb: ERROR Run u9296m2v errored: NameError("name 'model' is not defined")


(54000,)


wandb: Agent Starting Run: 2az877ko with config:
wandb: 	activation_function: Rectified_unit
wandb: 	batch_size: 1024
wandb: 	hidden_layers: 4
wandb: 	hidden_node_layer: 64
wandb: 	loss_function: SquaredError
wandb: 	max_epochs: 50
wandb: 	optimizer: adagrad_gd
wandb: 	rate: 0.001
wandb: 	weight_initialization: random


wandb: ERROR Run 2az877ko errored: NameError("name 'model' is not defined")
wandb: ERROR Detected 3 failed runs in the first 60 seconds, killing sweep.
wandb: To disable this check set WANDB_AGENT_DISABLE_FLAPPING=true


(54000,)


In [64]:
ans = raw_input('Enter: ')
if not ans:
    print("You entered nothing!")
else:
    print ("You entered something!")

NameError: ignored

In [123]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train,y_train,x_val,y_val=train_test_split(x_train, y_train, test_size=0.1, random_state=42)
wandb.log({'loss':np.mean(CE), 'trainingaccuracy':trainingaccuracy[epoch], 'validationaccuracy':validationaccuracy[epoch],'epoch':epoc

In [ ]:
nn_model=FNN(784,10,[10,20],"stochastic_gd",16,"sigmoid",
                                 "SquaredError",True,0.0001
                                 ,'random')
nn_model.optimization(10,nn_model,"stochastic_gd")
config_defaults = dict(
            max_epochs=5,
            hidden_layers=3,
            hidden_node_layer=32,
            rate=1e-3,
            optimizer="stochastic_gd",
            batch_size=16,
            weight_initialization="xavier",
            activation_function="Tanh",
            loss_function="CROSS",
        )

In [2]:
import numpy as np
l=[1,2,3]
print(np.mean(l))

2.0
